## Modularity
### H3: Random Hypothesis
* Randomly wired networks lack an inherent community structure.
* By comparing the link density of a community with the link density obtained for the same group of nodes for a randomly rewired network, we could decide if the original community corresponds to a dense subgraph, or its connectivity pattern emerged by chance.


### Modularity
*The "quality" of systematic deviations from a random configuration of a graph.*
* Allows us to rate communities and compare them

Consider a graph, G:
* $N$ nodes, $L$ links
* $n_c$ communities, each having $N_c$ nodes and $L_c$ links ($c = 1..n_c$)
* If $L_c > L_{expect}(C_c)$ then C_c is likely a community
* Measure difference between actual wiring diagram ($A_{ij}$) and the expected number of links between $i$ and $j$ in a random graph with wiring probability $p_{ij}$.

$M_c = \frac{1}{2L} \sum_{i,j \in C_c}{(A_{ij} - p_{ij})} = 
\frac{L_C}{L} - \left(\frac{k_c}{2L}\right)^2 $, 
where $p_{ij} = \frac{k_i k_j}{2L}$

* If $M_c > 0$ then the subgraph $C_c$ has more links than expected and represents a potential community.
* If $M_c = 0$ then the connectivity is what can be expected from a random graph, and nothing final can be concluded.
* If $M_c = 0$ then $C_c$ is not a community.

Modularity of a partition which breaks a graph into $n_c$ communities:
$M = \sum_{c=1}^{c_n}M_c$





* Explain the concept of modularity in your own words.
    * A measure that allows us to compare the quality of communities in a quantitative manner.

In [5]:
REPUBLICAN = "Republican"
DEMOCRATIC = "Democratic"

In [18]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import io, re, warnings, community
warnings.filterwarnings('ignore')

In [ ]:
# Install "community"
!pip install python-louvain

In [11]:
url_h115 = 'https://raw.githubusercontent.com/suneman/socialgraphs2018/master/files/data_US_congress/H115.csv'
df = pd.read_csv(url_h115)
page_names = list(df.WikiPageName)
dict_politician = dict(zip(df.WikiPageName, zip(df.Party, df.State)))

In [12]:
# A politcian class for storing data in a neat way.
class Politician:
    def __init__(self, wiki_name, party, state):
        self.WikiName = wiki_name
        self.Party = party
        self.State = state
    
    def to_string(self):
        return "TwitterName: %s, Party: %s, Sate: %s" % (self.WikiName, self.Party, self.State)
    
    def __hash__(self):
        return hash(self.WikiName)
    
    def __eq__(self, other):
        return (
                self.__class__ == other.__class__ and 
                self.WikiName == other.WikiName
               )

In [16]:
# Reads a WikiPage txt file, scans it for links to other Wikipages,
# and returns a list of the found links
def get_article_links(wiki_name, directory="h115"):
    # The regex pattern for recognizing links on the form [x | y]
    # and only capturing 'x'.
    article_pattern = r'\[\[([^\]]*?)(?:\|.*?)*\]\]'
    path = './%s/%s.txt' % (directory, wiki_name)
    article = io.open(path, 'r', encoding='utf-8').read()
    article_links = re.findall(article_pattern, article)
    article_links = [a.replace(' ', '_') for a in article_links]
    return article_links

In [19]:
DG = nx.DiGraph()

for a_wiki_name in dict_politician:
    # Create politician node (a)
    a_party, a_state = dict_politician[a_wiki_name]
    a_node = Politician(a_wiki_name, a_party, a_state)
    a_article_links = get_article_links(a_wiki_name)
    
    for b_wiki_name in a_article_links:
        if b_wiki_name in page_names:
            b_party, b_state = dict_politician[b_wiki_name]
            b_node = Politician(b_wiki_name, b_party, b_state)
            DG.add_edge(a_node, b_node)

### Consider the undirected version of the graph for the 115th house of representatives.
* Compute the modularity when you partition nodes based on their party. Modularity is described in the Network Science book, section 9.4). * * Use equation 9.12 in the book to calculate the modularity M of the parties-partitioning. Are the parties good community?


In [20]:
# Convert graph, extract nodes
UG = DG.to_undirected()
nodes = UG.nodes()

# Split nodes on party
repub_nodes = [node for node in nodes if node.Party == REPUBLICAN]
demo_nodes = [node for node in nodes if node.Party == DEMOCRATIC]

# Create subgraphs
repub_subgraph = UG.subgraph(repub_nodes)
demo_subgraph = UG.subgraph(demo_nodes)

$M_c = \frac{1}{2L} \sum_{i,j \in C_c}{(A_{ij} - p_{ij})} = 
\frac{L_C}{L} - \left(\frac{k_c}{2L}\right)^2 $, 
where $p_{ij} = \frac{k_i k_j}{2L}$

In [30]:
repub_part = community.best_partition(repub_subgraph)
demo_part = community.best_partition(demo_subgraph)

mod_repub = community.modularity(repub_part, repub_subgraph)
mod_demo = community.modularity(demo_part, demo_subgraph)
party_modularity = (mod_repub + mod_demo) / 2

print("Avg. party modularity:", party_modularity)

Avg. party modularity: 0.5419020430161754


Answer: Since the parties have an average $M_c$ value greater than zero, parties are potential communities

In [38]:
states = list(df.State)
state_modularity = []

for state in states:
    state_nodes = [node for node in nodes if node.State == state]
    state_subgraph = UG.subgraph(state_nodes)
    state_part = community.best_partition(state_subgraph)
    try:
        mod = community.modularity(state_part, state_subgraph)
        state_modularity.append(mod)
    except:
        print(state, "has no links.")

m = np.array(state_modularity).mean()
print("Mean state modularity:", m)

Alaska has no links.
Oregon has no links.
Connecticut has no links.
Mississippi has no links.
Oregon has no links.
Colorado has no links.
Connecticut has no links.
Idaho has no links.
Oregon has no links.
Rhode Island has no links.
Iowa has no links.
Connecticut has no links.
Colorado has no links.
Iowa has no links.
Colorado has no links.
Vermont has no links.
Colorado has no links.
Mississippi has no links.
Connecticut has no links.
Colorado has no links.
Oregon has no links.
Rhode Island has no links.
Arkansas has no links.
Idaho has no links.
South Dakota has no links.
Mississippi has no links.
Colorado has no links.
Arkansas has no links.
Oregon has no links.
Hawaii has no links.
North Dakota has no links.
Connecticut has no links.
Hawaii has no links.
Iowa has no links.
Colorado has no links.
Arkansas has no links.
Arkansas has no links.
Iowa has no links.
Mississippi has no links.
Delaware has no links.
Wyoming has no links.
Mean state modularity: 0.3621327528394268


* Repeat the exercise above by considering states instead of parties. Are the states good communities?


* Would you expect these results in light of what we have found in the previous exercises?

### Use the Python Louvain-algorithm implementation to find communities in the full house of representatives network. 
* Report the value of modularity found by the algorithm. 
* Is it higher or lower than what you found above for the parties/states as communities? 
* What does this comparison reveal about the parties/states?

In [39]:
full_part = community.best_partition(UG)
full_mod = community.modularity(full_part, UG)
full_mod

0.4552256058673469

Conclusion
* Slightly below the average for the parties, meaning parties is probably a better foundation for communities within the WikiPedia context, than just taking the "best" partitions in the full graph